# 1) Landsat-8 image search and visualization

    This code utilizes the EODataAccessGateway (EODAG) to search and filter satellite images based on specific criteria like cloud cover and date range within a defined geographic footprint, and then visualizes the area and the retrieved satellite image metadata on a map using Folium and GeoPandas. It starts by reading a shapefile to define the area of interest, sets search parameters for satellite imagery, filters results for Landsat 8 images, and finally, displays the selected image metadata and the area of interest on an interactive map.

In [ ]:
from eodag import EODataAccessGateway
import os
import folium
import geopandas as gpd
import pandas as pd
from IPython.display import display
import geojson

shapefile_path = '/path/to/area_boundaries/boundaries.shp'
footprint_data = gpd.read_file(shapefile_path)
coordinates = footprint_data.total_bounds

min_x = int(coordinates[0])
min_y = int(coordinates[1])
max_x = int(coordinates[2])
max_y = int(coordinates[3])

footprint = [min_x, min_y, max_x, max_y]
print('min_x:', min_x, 'min_y', min_y, 'max_x:', max_x, 'max_y:', max_y)

cloud_cover = 10
start = "2022-12-20"
end =   "2022-12-28"
provider = 'planetary_computer'

dag = EODataAccessGateway()

product_type = 'LANDSAT_C2L2'

def set_preferred_provider(self, provider):
        if provider not in self.available_providers():
            raise UnsupportedProvider(
                f"This provider is not recognised by eodag: {provider}"
            )
        preferred_provider, max_priority = self.get_preferred_provider()
        if preferred_provider != provider:
            new_priority = max_priority + 1
            self._plugins_manager.set_priority(provider, new_priority)

dag.set_preferred_provider(provider)

search_results, estimated_total_nbr_of_results = dag.search(
    productType=product_type,
    geom=footprint,
    start=start,
    end=end,
    cloudCover=cloud_cover,
)

search_results = [result for result in search_results if "landsat-8" in result.properties["platformSerialIdentifier"]]

print("Estimated total number of available images:", estimated_total_nbr_of_results)
print('Number of searched images:', len(search_results))

filtered_search_results = [result for result in search_results if result.properties["title"] in search_results]

image_properties_list = []
for result in search_results:
    properties = result.properties
    image_properties_list.append(properties)

image_df = pd.DataFrame(image_properties_list)
display(image_df)

geojson_features = []
for result in search_results:
    geometry = result.geometry
    properties = result.properties
    title = properties.get("title")
    geojson_feature = geojson.Feature(geometry=geometry, properties={"title": title})
    geojson_features.append(geojson_feature)

geojson_feature_collection = geojson.FeatureCollection(geojson_features)

emap = folium.Map([7, 3], zoom_start=7)
layer = folium.features.GeoJson(
    data=geojson_feature_collection,
    tooltip=folium.GeoJsonTooltip(fields=["title"]),
).add_to(emap)

shapefile_geojson = footprint_data.__geo_interface__
style = {'fillColor': 'yellow', 'color': 'yellow'} 
folium.GeoJson(shapefile_geojson, style_function=lambda x: style,).add_to(emap)
folium.GeoJson(shapefile_geojson).add_to(emap)

emap

# 2) Downloading selected scenes

    This code iterates through a collection of satellite image search results, downloading images that match a specified Worldwide Reference System (WRS) path to a designated directory, while providing feedback on the download status or reasons for skipping certain images.

In [ ]:
DEFAULT_DOWNLOAD_WAIT = 2
DEFAULT_DOWNLOAD_TIMEOUT = 10

output_directory = '/path/to/save/images/'

target_wrs_path = 190  

for result in search_results:
    try:
        if "landsat:wrs_path" in result.properties:
            wrs_path = result.properties["landsat:wrs_path"]
            if wrs_path == target_wrs_path:
                downloaded_path = result.download(
                    progress_callback=None,
                    wait=DEFAULT_DOWNLOAD_WAIT,
                    timeout=DEFAULT_DOWNLOAD_TIMEOUT,
                    outputs_prefix=output_directory)
                print(f"Downloaded: {downloaded_path}")
            else:
                print(f"Skipped: WRS Path ({wrs_path}) does not match target ({target_wrs_path})")
        else:
            print("Skipped: WRS Path not available in result properties")
    except Exception as e:
        print(f"Error downloading image: {str(e)}")

# 3) Resampling and Saving Satellite Images

    This Python code resamples Landsat 8 satellite images to match the spatial resolution and coordinate reference system of a reference image, saving the resampled images to a specified output directory. It utilizes libraries such as rioxarray for raster operations and rasterio for resampling methods, handling images in a memory-efficient manner with the inclusion of garbage collection. (Landsat-8  30m -> Sentinel-2 20m)

In [ ]:
import os
import numpy as np
import rioxarray
from rasterio.enums import Resampling
import gc

input_folder = '/path/to/images/'
output_folder = '/path/to/save/images_resampled/'
reference_image_path = '/path/to/reference_image.tif'

os.makedirs(output_folder, exist_ok=True)

reference_image = rioxarray.open_rasterio(reference_image_path)
    
for file_name in os.listdir(input_folder):
    if file_name.endswith(".TIF")and file_name.startswith("L8"): 
        input_path = os.path.join(input_folder, file_name)
        output_path = os.path.join(output_folder, file_name)

        input_image = rioxarray.open_rasterio(input_path)
        input_image = input_image.astype("float32")
        input_image.rio.write_nodata(np.nan)

        resampled_image = input_image.rio.reproject_match(reference_image, resampling=Resampling.bilinear)

        resampled_image.rio.to_raster(output_path)
        gc.collect() 

print("Resampling completed. Images saved in:", output_folder)

# 4) Automated Calculation and Saving of Various Satellite Image Indices

    This Python script automatically processes satellite images to calculate multiple vegetation and water indices, such as NDVI, GNDVI, and NDWI, among others, by reading specific band files, applying formulas to compute the indices, and then saving the results as TIFF files in a structured directory.

In [ ]:
import os
import numpy as np
import rasterio

folder_path = '/path/to/images_resampled/'
output_folder = '/path/to/save/indices/'

def calculate_all_indices(red_path, output_folder):
    try:
        print("_"*120)
        print(f"Processing {red_path}\n")
        
        base_name = os.path.splitext(os.path.basename(red_path))[0]
        
        nir_path = os.path.join(os.path.dirname(red_path), f"{base_name.replace('_SR_B4', '_SR_B5')}.TIF")
        green_path = os.path.join(os.path.dirname(red_path), f"{base_name.replace('_SR_B4', '_SR_B3')}.TIF")
        blue_path = os.path.join(os.path.dirname(red_path), f"{base_name.replace('_SR_B4', '_SR_B2')}.TIF")
        swir1_path = os.path.join(os.path.dirname(red_path), f"{base_name.replace('_SR_B4', '_SR_B6')}.TIF")
        swir2_path = os.path.join(os.path.dirname(red_path), f"{base_name.replace('_SR_B4', '_SR_B7')}.TIF")
        
        with rasterio.open(red_path) as red_src, \
                rasterio.open(green_path) as green_src, \
                rasterio.open(blue_path) as blue_src, \
                rasterio.open(nir_path) as nir_src, \
                rasterio.open(swir1_path) as swir1_src, \
                rasterio.open(swir2_path) as swir2_src:        
            
            red = red_src.read(1).astype(np.float32) / 65535.0 # max value of uint16
            nir = nir_src.read(1).astype(np.float32) / 65535.0  
            green = green_src.read(1).astype(np.float32) / 65535.0  
            blue = blue_src.read(1).astype(np.float32) / 65535.0  
            swir1 = swir1_src.read(1).astype(np.float32) / 65535.0
            swir2 = swir2_src.read(1).astype(np.float32) / 65535.0
        
        np.seterr(divide='ignore', invalid='ignore')
    
        red[red == 0] = np.nan
        nir[nir == 0] = np.nan
        green[green == 0] = np.nan
        blue[blue == 0] = np.nan
        swir1[swir1 == 0] = np.nan
        swir2[swir2 == 0] = np.nan 
    
        ndvi = (nir - red) / (nir + red)
        gndvi = (nir - green) / (nir + green)
        evi = 2.5 * ((nir - red) / (nir + 6 * red - 7.5 * blue + 1))
        savi = (1 + 0.5) * ((nir - red) / (nir + red + 0.5))
        osavi = ((nir - red) / (nir + red + 0.16))
        dvi = nir - red
        sr = red / nir
        gemi = ((2 * (nir ** 2 - red ** 2) + 1.5 * nir + 0.5 * red) / (nir + red + 0.5)) * (1 - 0.25 * ((2 * (nir ** 2 - red ** 2) + 1.5 * nir + 0.5 * red) / (nir + red + 0.5))) - ((red - 0.125) / (1 - nir))
        ndwi = (green - nir) / (green + nir)
        mndwi = (green - swir1) / (green + swir1)
        lswi = (nir - swir1) / (nir + swir1)
        ui = (swir2 - nir) / (swir2 + nir)
        ndbi = (swir1 - nir) / (swir1 + nir)
        mndbi = (swir2 - blue) / (swir2 + blue)      
        bsi = ((red + swir1) - (nir + blue)) / ((red + swir1) + (nir + blue))       
   
        index_date = base_name[17:25]
        path_row = base_name[10:16]
        formatted_date = f"{index_date[0:4]}-{index_date[4:6]}-{index_date[6:8]}_{path_row}"
    
        date_folder = os.path.join(output_folder, formatted_date)
        create_folder_if_not_exists(date_folder)
        
        indices = {'L8_NDVI': ndvi,'L8_GNDVI': gndvi,'L8_EVI': evi,'L8_SAVI': savi,'L8_OSAVI': osavi,'L8_DVI': dvi,'L8_SR': sr,'L8_GEMI': gemi,"L8_NDWI": ndwi,"L8_BSI": bsi,"L8_MNDBI": mndbi,"L8_LSWI": lswi,"L8_MNDWI": mndwi,"L8_NDBI": ndbi,"L8_UI": ui,}
        
        for index_name, index_value in indices.items():
            output_file = os.path.join(date_folder, f"{index_name}_{formatted_date}.tif")
            save_index_as_tiff(index_value, output_file, red_path)
            print(f"Calculated {index_name} and saved to: {output_file}")
            
    except Exception as e:
        print(f"An error occurred while processing {red_path}: {e}")
    
def save_index_as_tiff(index, output_file, red_path):
    with rasterio.open(red_path) as src:
        transform = src.transform
        crs = src.crs
    with rasterio.open(output_file, 'w', driver='GTiff', width=index.shape[1], height=index.shape[0],
                       count=1, dtype=index.dtype, crs=crs, transform=transform) as dst:
        dst.write(index, 1)

def create_folder_if_not_exists(folder_path):
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

def process_folder_and_subfolders(folder_path):
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith("_B4.TIF"):
                red_path = os.path.join(root, file)

                calculate_all_indices(red_path, output_folder)

process_folder_and_subfolders(folder_path)

print()
print("All files saved :)")